In [ ]:
#average-mean-tabpfn

import os
import pandas as pd

# Path to the folder with CSV files
pasta = ""
arquivos = [f for f in os.listdir(pasta) if f.endswith('.csv')]

# List to store all DataFrames
dfs = []

# Reads and accumulates data
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)
    df = pd.read_csv(caminho_arquivo)
    dfs.append(df)

# Concatenate all files into a single DataFrame
df_total = pd.concat(dfs, ignore_index=True)

# Checks if 'Threshold' column is present
if 'threshold' not in df_total.columns:
    raise ValueError("A coluna 'Threshold' não foi encontrada nos arquivos CSV.")

# Remove non-numeric columns except 'Threshold'
colunas_numericas = df_total.select_dtypes(include='number').columns.tolist()
if 'threshold' not in colunas_numericas:
    colunas_numericas.insert(0, 'threshold')

df_total = df_total[colunas_numericas]

# Group by 'Threshold' and calculate mean and standard deviation
media = df_total.groupby('threshold').mean()
desvio = df_total.groupby('threshold').std()

# Adds column indicating the type of statistic
media['fold'] = 'média'
desvio['fold'] = 'desvio_padrão'

# Merge results vertically
resultado = pd.concat([media, desvio], axis=0).reset_index()

# Save the result
resultado.to_csv(os.path.join(pasta, 'mlp_oversampling_media_desvio.csv'), index=False)

In [ ]:
#average-mean-tabpfn

import os
import pandas as pd


arquivos = [
    "../MODELOS-TRANSFORMERS_RESULTADOS/Resultados_TabPFN/6h/test/resultados_TabPFN_Test_fold0.csv",
    "../MODELOS-TRANSFORMERS_RESULTADOS/Resultados_TabPFN/6h/test/resultados_TabPFN_Test_fold1.csv",
    "../MODELOS-TRANSFORMERS_RESULTADOS/Resultados_TabPFN/6h/test/resultados_TabPFN_Test_fold2.csv",
    "../MODELOS-TRANSFORMERS_RESULTADOS/Resultados_TabPFN/6h/test/resultados_TabPFN_Test_fold3.csv",
    "../MODELOS-TRANSFORMERS_RESULTADOS/Resultados_TabPFN/6h/test/resultados_TabPFN_Test_fold4.csv"
]


for caminho in arquivos:
    if not os.path.exists(caminho):
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")

dfs = []

for i, caminho_arquivo in enumerate(arquivos):
    df = pd.read_csv(caminho_arquivo)
    df['fold'] = i
    dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)

if 'Threshold' not in df_total.columns:
    raise ValueError("A coluna 'Threshold' não foi encontrada nos arquivos CSV.")

colunas_numericas = df_total.select_dtypes(include='number').columns.tolist()
colunas_para_estatistica = ['Threshold'] + [col for col in colunas_numericas if col != 'fold']

media = df_total[colunas_para_estatistica].groupby('Threshold').mean().reset_index()
desvio = df_total[colunas_para_estatistica].groupby('Threshold').std().reset_index()

media['fold'] = 'média'
desvio['fold'] = 'desvio_padrão'

colunas_finais = df_total.columns.tolist()
media = media[colunas_finais]
desvio = desvio[colunas_finais]

resultado_final = pd.concat([df_total, media, desvio], ignore_index=True)

pasta_saida = os.path.dirname(arquivos[0])

resultado_final.to_csv(os.path.join(pasta_saida, '../MODELOS-TRANSFORMERS_RESULTADOS/Resultados_Teste_Versao2/TabPFN.csv'), index=False)


In [ ]:
# === Statistical Test ===

import pandas as pd
from scipy.stats import friedmanchisquare, f
import scikit_posthocs as sp

# === Config ===
csv_file = "../all_models_statistics_B7.csv"
model_column = "Model"
fold_column = "Fold"
metrics = ["TSS", "HSS", "AUC", "TPR", "TNR", "FPR", "FNR"]
output_csv = "../friedman_iman_nemenyi_holm_results_B7.csv"

# === Read CSV ===
df = pd.read_csv(csv_file, sep=";")

rows = []

for metric in metrics:
    print(f"\n--- Metric: {metric} ---")

    # Reshape into matrix format: each column = model, each row = fold
    table = df.pivot(index=fold_column, columns=model_column, values=metric)

    # Apply Friedman test
    stat, p = friedmanchisquare(*[table[model] for model in table.columns])

    # Parameters
    N = table.shape[0]  # number of folds
    k = table.shape[1]  # number of models

    # Compute Iman–Davenport statistic
    F_F = ((N - 1) * stat) / (N * (k - 1) - stat)
    df1 = k - 1
    df2 = (k - 1) * (N - 1)
    p_davenport = f.sf(F_F, df1, df2)  # p-value

    # Compute Nemenyi post-hoc test
    nemenyi = sp.posthoc_nemenyi_friedman(table.values)
    nemenyi.index = table.columns
    nemenyi.columns = table.columns

    # Compute Dunn-Holm post-hoc test
    holm_results = sp.posthoc_dunn(table.T.values, p_adjust='holm')
    holm_results.index = table.columns
    holm_results.columns = table.columns

    # Generate only unique model comparisons (Model1 < Model2)
    models = table.columns.tolist()
    for i in range(len(models)):
        for j in range(i + 1, len(models)):
            model1 = models[i]
            model2 = models[j]

            p_nemenyi = nemenyi.loc[model1, model2]
            p_holm = holm_results.loc[model1, model2]

            rows.append({
                "Metric": metric,
                "Friedman_Stat": stat,
                "p_value_Friedman": p,
                "ImanDavenport_Stat": F_F,
                "p_value_ImanDavenport": p_davenport,
                "Model1": model1,
                "Model2": model2,
                "p_value_Nemenyi": p_nemenyi,
                "p_value_Holm": p_holm
            })

# Save all results to CSV
pd.DataFrame(rows).to_csv(output_csv, index=False, float_format="%.8f")

print(f"\nFile generated: {output_csv}")




--- Métrica: TSS ---

--- Métrica: HSS ---

--- Métrica: AUC ---

--- Métrica: TPR ---

--- Métrica: TNR ---

--- Métrica: FPR ---

--- Métrica: FNR ---

Arquivo gerado: ../MODELOS-TRANSFORMERS_RESULTADOS/experimentos_nowcasting/testes-estatisticos/resultados_friedman_iman_nemenyi_holm_pairs_unique_B7.csv
